This is an example for the following question on Quantum StackExchange: [Why do I get positive amplitudes when I create a qiskit StateVectorCircuit from only negative amplitudes?](https://quantumcomputing.stackexchange.com/questions/12534/why-do-i-get-positive-amplitudes-when-i-create-a-qiskit-statevectorcircuit-from)

In the first part I will you show my usecase and in the second part I have a small example, which I have used in the question.

In the following I prepare the breast cancer dataset for amplitude encoding. Here I will have amplitudes between -1 and 1 without an imaginary part.

The resulting training set has the following properties:
* the first vector has only negative relative phases -> a global negative phase
* the second vector has different relative phases, but the first one is positive

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

from qiskit.aqua.circuits import StateVectorCircuit
from qiskit import *
from qiskit.circuit.library import *

data, target = datasets.load_breast_cancer(return_X_y=True)
sample_train, sample_test, label_train, label_test = train_test_split(data, target, test_size=0.3, random_state=12)

std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

reshaped_sample_train = []

for i in range(len(sample_train)): # Pad to length of 32 (2**5)
    reshaped_sample_train.append(sample_train[i].tolist())
    reshaped_sample_train[i].append(0)
    reshaped_sample_train[i].append(0)
    # TODO convert to amplitude_vector
    
reshaped_sample_train = np.array(reshaped_sample_train)

reshaped_sample_train_normalized = preprocessing.normalize(reshaped_sample_train, norm='l2')

print(len(reshaped_sample_train[0]))

32


In [2]:
state_vector_circuit = StateVectorCircuit(reshaped_sample_train_normalized[1]).construct_circuit()

job = execute(state_vector_circuit, Aer.get_backend('statevector_simulator'), optimization_level=0)

result = job.result()

outputstate = result.get_statevector(state_vector_circuit, decimals=8)
print('Result from the statevector_simulator (second vector):\n' + str(outputstate))
print('Input of the StateVectorCircuit (second vector):\n' + str(reshaped_sample_train_normalized[1]))

Result from the statevector_simulator (second vector):
[ 0.00681613+0.j  0.08419017+0.j  0.00562091+0.j -0.10176068-0.j
 -0.05114783+0.j -0.11014482-0.j -0.07194248+0.j -0.06184889+0.j
 -0.1198638 -0.j -0.21566368-0.j -0.28117491+0.j -0.24055868+0.j
 -0.28190676+0.j -0.30325892+0.j -0.28027806+0.j -0.25745632+0.j
 -0.28637488+0.j -0.20404273+0.j -0.30764855+0.j -0.3117047 +0.j
 -0.01139919+0.j  0.14565393-0.j -0.01895732+0.j -0.13545555+0.j
 -0.0315545 +0.j -0.11372543+0.j  0.00781869-0.j  0.12463429-0.j
 -0.1567995 +0.j -0.18717026+0.j  0.        +0.j  0.        +0.j]
Input of the StateVectorCircuit (second vector):
[ 0.00681613  0.08419017  0.00562091 -0.10176068 -0.05114783 -0.11014482
 -0.07194248 -0.06184889 -0.1198638  -0.21566368 -0.28117491 -0.24055868
 -0.28190676 -0.30325892 -0.28027806 -0.25745632 -0.28637488 -0.20404273
 -0.30764855 -0.3117047  -0.01139919  0.14565393 -0.01895732 -0.13545555
 -0.0315545  -0.11372543  0.00781869  0.12463429 -0.1567995  -0.18717026
  0.      

In [3]:
state_vector_circuit = StateVectorCircuit(reshaped_sample_train_normalized[0]).construct_circuit()

job = execute(state_vector_circuit, Aer.get_backend('statevector_simulator'), optimization_level=0)

result = job.result()

outputstate = result.get_statevector(state_vector_circuit, decimals=8)
print('Result from the statevector_simulator (first vector):\n' + str(outputstate))
print('Input of the StateVectorCircuit (first vector):\n' + str(reshaped_sample_train_normalized[0]))

Result from the statevector_simulator (first vector):
[0.1564843 +0.j 0.16922211+0.j 0.16232775-0.j 0.20597463-0.j
 0.03423234+0.j 0.19600826+0.j 0.22737921+0.j 0.19004087+0.j
 0.12500511+0.j 0.07550138+0.j 0.23795168+0.j 0.09240364+0.j
 0.2407529 +0.j 0.24396496+0.j 0.02540669+0.j 0.18287985+0.j
 0.23692859-0.j 0.10234472-0.j 0.09063713+0.j 0.18696281+0.j
 0.19347473-0.j 0.19026994-0.j 0.20068004-0.j 0.22913517-0.j
 0.08538835-0.j 0.22834957-0.j 0.23960735-0.j 0.18002142-0.j
 0.22172094-0.j 0.19501441-0.j 0.        -0.j 0.        -0.j]
Input of the StateVectorCircuit (first vector):
[-0.1564843  -0.16922211 -0.16232775 -0.20597463 -0.03423234 -0.19600826
 -0.22737921 -0.19004087 -0.12500511 -0.07550138 -0.23795168 -0.09240364
 -0.2407529  -0.24396496 -0.02540669 -0.18287985 -0.23692859 -0.10234472
 -0.09063713 -0.18696281 -0.19347473 -0.19026994 -0.20068004 -0.22913517
 -0.08538835 -0.22834957 -0.23960735 -0.18002142 -0.22172094 -0.19501441
  0.          0.        ]


Here is a simpler example, which I used in the Question

In [4]:
state_vector = [-1/2, -1/2, -1/2, -1/2]

state_vector_circuit = StateVectorCircuit(state_vector).construct_circuit()

job = execute(state_vector_circuit, Aer.get_backend('statevector_simulator'), optimization_level=0)
result = job.result()

outputstate = result.get_statevector(state_vector_circuit)

print('Result from the statevector_simulator: ' + str(outputstate))
print('Input of the StateVectorCircuit:       ' + str(state_vector))

Result from the statevector_simulator: [0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
Input of the StateVectorCircuit:       [-0.5, -0.5, -0.5, -0.5]


In [5]:
state_vector = [-1/2, 1/2, 1/2, -1/2]

state_vector_circuit = StateVectorCircuit(state_vector).construct_circuit()

job = execute(state_vector_circuit, Aer.get_backend('statevector_simulator'), optimization_level=0)
result = job.result()

outputstate = result.get_statevector(state_vector_circuit)

print('Result from the statevector_simulator: ' + str(outputstate))
print('Input of the StateVectorCircuit:       ' + str(state_vector))

Result from the statevector_simulator: [ 0.5+0.000000e+00j -0.5-6.123234e-17j -0.5-6.123234e-17j
  0.5+0.000000e+00j]
Input of the StateVectorCircuit:       [-0.5, 0.5, 0.5, -0.5]
